# Python-based pipeline for preprocessing of RNA-seq datasets

## Protocol B-1 - Data Pre-processing for RNA-seq dataset

## Step 1. Set-up

### Install and import libraries

Install the packages using `pip`.
If you want to install the library, then type  "pip3 install `package name`" in terminal.

In [ ]:
import os, math, shutil, time, datetime, importlib

libraries = {
    'numpy': 'numpy',
    'Bio': 'biopython',
    'pysam': 'pysam',
    'pandas': 'pandas'
}

# Check and install Python libraries
for lib, package_name in libraries.items():
    try:
        importlib.import_module(lib)
        print(f"{lib} is installed.")
    except ImportError:
        print(f"{lib} is not installed. Installing {package_name}...")
        os.system(f'pip3 install {package_name}')

# Binary tools to check and install
binary_tools = ['samtools', 'bedtools', 'bowtie2']

# Check and install binary tools
for tool_name in binary_tools:
    if os.system(f'which {tool_name} > /dev/null 2>&1') == 0:
        print(f"{tool_name} is already installed.")
    else:
        print(f"{tool_name} is not installed. Installing {tool_name}...")
        os.system(f'apt-get update && apt-get install -y {tool_name}')

numpy is installed.
Bio is not installed. Installing biopython...
pysam is not installed. Installing pysam...
pandas is installed.
samtools is not installed. Installing samtools...
bedtools is not installed. Installing bedtools...
bowtie2 is not installed. Installing bowtie2...


In [ ]:
from numpy import zeros
from Bio import SeqIO
import pandas as pd
import pysam

### Function

Please run the cell without modification

In [ ]:
def count_coverage(samfile, chromosome_size=6000000, flip=True):
    """counts coverage per base in a strand-specific manner

    For paired-end reads, the insert betwen the mapped reads is
    also counted.

    flip: Whether or not the strands should be flipped.
    This should be true for RNA-seq, and false for ChIP-exo

    chromosome_size: This value should be larger than the largest chromosome"""

    if samfile.has_index():
        return count_coverage_indexed(samfile, chromosome_size=chromosome_size, flip=flip)
    all_counts = {}
    plus_strands = []
    minus_strands = []
    if "SQ" in samfile.header:
        chromosome_sizes = {}
        for entry in samfile.header["SQ"]:
            chromosome_sizes[entry["SN"]] = int(entry["LN"]) + 1
    else:
        for reference in samfile.references:
            chromosome_sizes[reference] = chromosome_size
    for reference in samfile.references:  # create an array for each reference
        plus_strands.append(zeros((chromosome_sizes[reference],)))
        minus_strands.append(zeros((chromosome_sizes[reference],)))
    # iterate through each mapped read
    for i, read in enumerate(samfile):
        if read.is_unmapped:
            continue
        if not read.is_proper_pair:
            if read.is_reverse:
                minus_strands[read.tid][read.pos:read.aend] += 1
            else:
                plus_strands[read.tid][read.pos:read.aend] += 1
        # for paired-end data, get entire insert from only read1
        elif read.is_read1:
            if read.is_reverse:
                minus_strands[read.tid][read.pnext:read.aend] += 1
            else:
                plus_strands[read.tid][read.pos:read.pos + read.isize] += 1
    # store the results per reference
    for i, reference in enumerate(samfile.references):
        all_counts[reference] = {}
        if flip:
            all_counts[reference]["-"] = plus_strands[i]
            all_counts[reference]["+"] = minus_strands[i]
        else:
            all_counts[reference]["+"] = plus_strands[i]
            all_counts[reference]["-"] = minus_strands[i]
    return all_counts

def count_coverage_indexed(samfile, chromosome_size=6000000, flip=True):
    """counts coverage per base in a strand-specific manner

    For paired-end reads, the insert betwen the mapped reads is
    also counted.

    flip: Whether or not the strands should be flipped.
    This should be true for RNA-seq, and false for ChIP-exo

    chromsome_size: This value should be larger than the largest chromosome"""
    if "SQ" in samfile.header:
        chromosome_sizes = {}
        for entry in samfile.header["SQ"]:
            chromosome_sizes[entry["SN"]] = int(entry["LN"]) + 1
    else:
        for reference in samfile.references:
            chromosome_sizes[reference] = chromosome_size
    all_counts = {}
    for reference in samfile.references:  # go through each chromosome
        plus_strand = zeros((chromosome_sizes[reference],))
        minus_strand = zeros((chromosome_sizes[reference],))
        # iterate through each mapped read
        for i, read in enumerate(samfile.fetch(reference=reference)):
            if not read.is_proper_pair:
                if read.is_reverse:
                    minus_strand[read.pos:read.aend] += 1
                else:
                    plus_strand[read.pos:read.aend] += 1
            # for paired-end data, get entire insert from only read1
            elif read.is_read1:
                if read.is_reverse:
                    minus_strand[read.pnext:read.aend] += 1
                else:
                    plus_strand[read.pos:read.pos + read.isize] += 1
            all_counts[reference] = {}
            if flip:
                all_counts[reference]["-"] = plus_strand
                all_counts[reference]["+"] = minus_strand
            else:
                all_counts[reference]["+"] = plus_strand
                all_counts[reference]["-"] = minus_strand
    return all_counts

def write_samfile_to_gff(samfile, output, chromosome_size=6000000,
                         separate_strand=True, flip=True, log_scale=True):
    """write samfile object to an output object in a gff format

    flip: Whether or not the strands should be flipped.
    This should be true for RNA-seq, and false for ChIP-exo

    chromsome_size: This value should be larger than the largest chromosome

    separate_strand: Whether the forward and reverse strands should be made
    into separate tracks (True) or the negative strand should be rendered
    as negative values (False)
    """
    all_counts = count_coverage(samfile, chromosome_size=chromosome_size,
                                flip=flip)

    name = os.path.split(samfile.filename)[1].decode()
    for reference in all_counts:
        for strand in all_counts[reference]:
            counts = all_counts[reference][strand]
            for i in counts.nonzero()[0]:
                if log_scale:
                    count = math.log(float(counts[i]), 2)
                else:
                    count = counts[i]
                if separate_strand:
                    output.write("%s\t\t%s\t%d\t%d\t%.2f\t%s\t.\t.\n" %
                        (reference, "%s_(%s)" % (name, strand), i, i,count, strand))
                else:
                    output.write("%s\t\t%s\t%d\t%d\t%.2f\t%s\t.\t.\n" %
                        (reference, name, i, i, count,strand))

def convert_samfile_to_gff(sam_filename, out_filename, chromosome_size=6000000,
                           separate_strand=True, flip=True, log_scale=True):
    """read in the a samfile from a path, and write it out to a gff filepath

    flip: Whether or not the strands should be flipped.
    This should be true for RNA-seq, and false for ChIP-exo.

    chromsome_size: This value should be larger than the largest chromosome.

    separate_strand: Whether the forward and reverse strands should be made
    into separate tracks (True) or the negative strand should be rendered
    as negative values (False)
    """
    samfile = pysam.Samfile(sam_filename)
    with open(out_filename, "w") as outfile:
        write_samfile_to_gff(samfile, outfile, chromosome_size=chromosome_size,
                             separate_strand=separate_strand, flip=flip,
                             log_scale=log_scale)
    samfile.close()

def convert_sam_to_bedgraph(sam_bam_sorted, dir_bed_output):
    bedgraph_file_positive = f"{dir_bed_output}_positive.bedgraph"
    bedgraph_file_negative = f"{dir_bed_output}_negative.bedgraph"

    !bedtools genomecov -ibam $sam_bam_sorted -bg -strand + > $bedgraph_file_positive
    !bedtools genomecov -ibam $sam_bam_sorted -bg -strand - > $bedgraph_file_negative

### (Optional) Google Drive Mount

If using Google Colab, it is necessary to mount Google Drive to access the files stored within it.

In [ ]:
from google.colab import drive, files

drive.mount('/content/mnt')

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


### Set the working directory

In [ ]:
working_directory =  '/content/mnt/My Drive/Test/RNA-seq'  #Enter your working directory.

os.chdir(working_directory)
print(f'Current working directiory : {os.getcwd()}')

Current working directiory : /content/mnt/My Drive/Test/RNA-seq


## Step 2. Prepare the files

Prepare the following files and move them into the working directory:
* **GenBank_file** : Enter the GenBank file name of the reference strain.
* **Organism** : Enter the accession number of the reference genome from the GenBank file. (e.g. *Escherichia coli* K-12 MG1655 : NC_000913.3)
* **Sample_name_list** : Enter the names of the prepared RNA-seq files, which include both R1 and R2 for paired-end sequencing, excluding the read number and file format (e.g., `RNA_seq_sample_1_R1.fastq.gz` and `RNA_seq_sample_1_R2.fastq.gz` -> `RNA_seq_sample_1`).

In [ ]:
GenBank_file = 'NC_000913.3.gb'
Organism = 'NC_000913.3'
Sample_name_list = ['mini_RNA-seq_Ecoli_mid-37-1',
                    'mini_RNA-seq_Ecoli_mid-37-2',
                    'mini_RNA-seq_Ecoli_del_rpoS_mid-37-1',
                    'mini_RNA-seq_Ecoli_del_rpoS_mid-37-2']

### Directory setting

In [ ]:
!mkdir '0_raw' '1_reference' '1_reference/rRNA_index' '1_reference/reference_genome_index' '2_alignment' '2_alignment/rRNA_del_fastq' '3_result'
!mv ./*.gb ./1_reference/
!mv ./*.gz ./0_raw/
print(f'Make directory : Success')

Make directory : Success


## Step 3. Generate Fasta files from the reference genome file

In [ ]:
# Set the name of reference files.
dir_genbank_file = f'./1_reference/{GenBank_file}'
dir_genome_fasta = f'./1_reference/{Organism}.fasta'
dir_rRNA_fasta = f'./1_reference/{Organism}_rRNA.fasta'

# Make the reference genome fasta file.
with open(dir_genome_fasta, "w") as genome_fasta:
    for record in SeqIO.parse(dir_genbank_file, "genbank"):
        SeqIO.write(record, genome_fasta, "fasta")

# Make the ribosomal RNA of reference genome fasta file.
with open(dir_rRNA_fasta, "w") as rRNA_fasta:
    for record in SeqIO.parse(dir_genbank_file, "genbank"):
        for feature in record.features:
            if feature.type == "rRNA":
                rRNA_seq = feature.extract(record)
                rRNA_seq.id = Organism
                rRNA_seq.description = ""
                SeqIO.write(rRNA_seq, rRNA_fasta, "fasta")

print('Step 3 is done.')

Step 3 is done.


## Step 4. Remove the rRNA sequences for RNA-seq dataset

Unzip and align the sequencing reads file (.fastq.gz) to the rRNA sequences from reference genome using Bowtie 2 and remove the aligned rRNA reads from each sequencing read.

In [ ]:
# Build the bowtie2 index set for removing the rRNA sequences
dir_rRNA_ref = f'./1_reference/rRNA_index/{Organism}'
!bowtie2-build $dir_rRNA_fasta $dir_rRNA_ref

#Data pre-process
for file in Sample_name_list:
    print(f'{file} processing ...')

    raw_R1_gz = f'./0_raw/{file}_R1.fastq.gz'
    raw_R2_gz = f'./0_raw/{file}_R2.fastq.gz'

    # Unzip the fastq.gz file
    !gunzip -k $raw_R1_gz > /dev/null
    !gunzip -k $raw_R2_gz > /dev/null

    raw_R1_fastq = f'./0_raw/{file}_R1.fastq'
    raw_R2_fastq = f'./0_raw/{file}_R2.fastq'

    rRNA_del_fastq_gz = f'./2_alignment/rRNA_del_fastq/rRNA_del_{file}.fastq.gz'
    rRNA_mapped_sam = f'./2_alignment/rRNA_mapped_{file}.sam'

    print('\t Aligning to rRNA sequences of reference genome...')
    !bowtie2 -x $dir_rRNA_ref -1 $raw_R1_fastq -2 $raw_R2_fastq --un-conc-gz $rRNA_del_fastq_gz -S $rRNA_mapped_sam
    !rm $rRNA_mapped_sam

print('Step 4 is done.')

Settings:
  Output files: "./1_reference/rRNA_index/NC_000913.3.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ./1_reference/NC_000913.3_rRNA.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 8020
Using parameters --bmax 6015 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 601

## Step 5. Align the RNA-seq reads to reference genome sequence

Align the rRNA-removed sequencing reads from step 3 to reference genome using Bowtie 2, with a maximum insert size of 1000 bp, allowing 1 mismatch after trimming 3 bp at 3' ends.

In [ ]:
# Build the bowtie2 index set of reference genome
dir_genome_ref = f'./1_reference/reference_genome_index/{Organism}'
!bowtie2-build $dir_genome_fasta $dir_genome_ref

for file in Sample_name_list:
    print(f'{file} processing ...')

    rRNA_del_R1_fastq_gz = f'./2_alignment/rRNA_del_fastq/rRNA_del_{file}.fastq.1.gz'
    rRNA_del_R2_fastq_gz = f'./2_alignment/rRNA_del_fastq/rRNA_del_{file}.fastq.2.gz'
    re_rRNA_del_R1_fastq_gz = f'./2_alignment/rRNA_del_fastq/rRNA_del_{file}_R1.fastq.gz'
    re_rRNA_del_R2_fastq_gz = f'./2_alignment/rRNA_del_fastq/rRNA_del_{file}_R2.fastq.gz'

    os.rename(rRNA_del_R1_fastq_gz,re_rRNA_del_R1_fastq_gz)
    os.rename(rRNA_del_R2_fastq_gz,re_rRNA_del_R2_fastq_gz)

    !gunzip -k $re_rRNA_del_R1_fastq_gz > /dev/null
    !gunzip -k $re_rRNA_del_R2_fastq_gz > /dev/null

    R1_file = re_rRNA_del_R1_fastq_gz.split(".gz")[0]
    R2_file = re_rRNA_del_R2_fastq_gz.split(".gz")[0]

    genome_mapped_sam = f'./2_alignment/genome_mapped_{file}.sam'

    print('\t Aligning to reference genome...')
    !bowtie2 -x $dir_genome_ref -1 $R1_file -2 $R2_file -S $genome_mapped_sam -X 1000 -N 1 -3 3

print('Step 5 is done.')

Settings:
  Output files: "./1_reference/reference_genome_index/NC_000913.3.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ./1_reference/NC_000913.3.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 1160413
Using parameters --bmax 870310 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters

## Step 6. Convert the SAM files generated from Bowtie 2 into BAM, GFF, and Bedgraph files

Convert the SAM files generated from Bowtie 2 into BAM using Samtools and generate GFF files for downstream analysis.

In [ ]:
for file in Sample_name_list:
    print(f'{file} processing ...')
    sam_bam_unsorted = f'./2_alignment/{file}.unsorted.bam'
    sam_bam_sorted = f'./2_alignment/{file}.bam'
    genome_mapped_sam = f'./2_alignment/genome_mapped_{file}.sam'

    !samtools view -bS $genome_mapped_sam -o $sam_bam_unsorted
    !samtools sort $sam_bam_unsorted -o $sam_bam_sorted
    !rm $sam_bam_unsorted
    !samtools index $sam_bam_sorted

    print("\t Samtools running is done.")

    dir_gff_output = f'./3_result/{file}.gff'
    convert_samfile_to_gff( genome_mapped_sam, dir_gff_output )
    print("\t Making the gff file is done.")

    dir_bed_output = f'./3_result/{file}'
    convert_sam_to_bedgraph(genome_mapped_sam, dir_bed_output)
    print('\t Making the bedGrpah file is done.\n')

print('Step 6 is done.')

mini_RNA-seq_Ecoli_mid-37-1 processing ...
	 Samtools running is done.
	 Making the gff file is done.
	 Making the bedGrpah file is done.

mini_RNA-seq_Ecoli_mid-37-2 processing ...
	 Samtools running is done.
	 Making the gff file is done.
	 Making the bedGrpah file is done.

mini_RNA-seq_Ecoli_del_rpoS_mid-37-1 processing ...
	 Samtools running is done.
	 Making the gff file is done.
	 Making the bedGrpah file is done.

mini_RNA-seq_Ecoli_del_rpoS_mid-37-2 processing ...
	 Samtools running is done.
	 Making the gff file is done.
	 Making the bedGrpah file is done.

Step 6 is done.
